In [0]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 2.7MB/s 


In [0]:
!pip install pytorch-nlp

     |████████████████████████████████| 92kB 2.2MB/s 


In [0]:
#import package
import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from torchnlp.datasets import imdb_dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

In [0]:
#-----------------fake.csv dataset
pd.set_option('display.max_columns', None)
df = pd.read_csv("fake.csv")

df = df[['text', 'type']]
print(df.head())

                                                text  type
0  Print They should pay all the back all the mon...  bias
1  Why Did Attorney General Loretta Lynch Plead T...  bias
2  Red State : \nFox News Sunday reported this mo...  bias
3  Email Kayla Mueller was a prisoner and torture...  bias
4  Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...  bias


In [0]:
df.tail

<bound method NDFrame.tail of                                                     text  type
0      Print They should pay all the back all the mon...  bias
1      Why Did Attorney General Loretta Lynch Plead T...  bias
2      Red State : \nFox News Sunday reported this mo...  bias
3      Email Kayla Mueller was a prisoner and torture...  bias
4      Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...  bias
...                                                  ...   ...
12994  It DOES allow you to put a dog face on top of ...    bs
12995  Wait till you see what happens to the valuatio...    bs
12996  I'm waiting for the one that puts a pussy on m...    bs
12997  $4 Billion even after they are known to be kee...    bs
12998  of course - how else would they disceminate te...    bs

[12999 rows x 2 columns]>

In [0]:
df.loc[df["type"] == 'state']

,text,type
7491,This handout picture released by the Venezuela...,state
7492,US air strikes in Afghanistan increase in 2016...,state
7493,Lebanon Lebanese Foreign Minister Gebran Bassi...,state
7494,Le terrorisme régional vise à protéger les int...,state
7495,Military An American soldier talks with Saudi ...,state
...,...,...
7607,Iraq Civilians leave their homes as Iraqi troo...,state
7608,Syria This file photo shows smoke rising from ...,state
7609,Pakistan This photo taken in Lahore on October...,state
7610,"FMs of Iran, Syria, Russia meet in Moscow Fri ...",state


In [0]:
print(df[0][1])

In [0]:
from collections import Counter
print(Counter(df['type'].values))

Counter({'bs': 11492, 'bias': 443, 'conspiracy': 430, 'hate': 246, 'satire': 146, 'state': 121, 'junksci': 102, 'fake': 19})


In [0]:
#change all 'type' in '1' (fake)
df.loc[df['type']=='bias', 'type'] = '1'
df.loc[df['type']=='bs', 'type'] = '1'
df.loc[df['type']=='conspiracy', 'type'] = '1'
df.loc[df['type']=='hate', 'type'] = '1'
df.loc[df['type']=='satire', 'type'] = '1'
df.loc[df['type']=='state', 'type'] = '1'
df.loc[df['type']=='junksci', 'type'] = '1'
df.loc[df['type']=='fake', 'type'] = '1'

print(df)
print(df.iloc[[400]])

                                                    text type
0      Print They should pay all the back all the mon...    1
1      Why Did Attorney General Loretta Lynch Plead T...    1
2      Red State : \nFox News Sunday reported this mo...    1
3      Email Kayla Mueller was a prisoner and torture...    1
4      Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...    1
...                                                  ...  ...
12994  It DOES allow you to put a dog face on top of ...    1
12995  Wait till you see what happens to the valuatio...    1
12996  I'm waiting for the one that puts a pussy on m...    1
12997  $4 Billion even after they are known to be kee...    1
12998  of course - how else would they disceminate te...    1

[12999 rows x 2 columns]
                                                  text type
400  22 Shares\n1 20 0 1\nThe anarchist political o...    1


In [0]:
from collections import Counter
print(Counter(df['type'].values))

Counter({'1': 12999})


In [0]:
#separate fake.csv file: in train.csv and test.csv
import pandas as pd
from sklearn.model_selection import train_test_split
data = df
data.head()

,text,type
0,Print They should pay all the back all the mon...,1
1,Why Did Attorney General Loretta Lynch Plead T...,1
2,Red State : \nFox News Sunday reported this mo...,1
3,Email Kayla Mueller was a prisoner and torture...,1
4,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,1


In [0]:
#12999 row and 2 columns
print(data.shape)

(12999, 2)


In [0]:
X = data
y = data

In [0]:
#test 20%(780), training 80%(12219)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("\nX_training:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)


X_training:

                                                    text type
3925   Report Copyright Violation Trump: No Salary fo...    1
3809   Your News Wire WikiLeaks Bombshell: ‘There Is ...    1
11177    reports on What You Need to Know About the '...    1
95     The second video (bottom one ) proves what Tru...    1
2140   “Chapo Trap House”: New Left-Wing Podcast is a...    1
(10399, 2)

X_test:

                                                    text type
1816   “I am a person of faith. I am a Christian. I a...    1
4383   Gardasil Vaccine Given without Consent and Rui...    1
5797   No Search Warrant Yet: FBI Can’t Move On Email...    1
4406   Only 3 Countries Left Without a ROTHSCHILD Cen...    1
12103  Waking Times \n“Does anyone even know why we e...    1
(2600, 2)


In [0]:
# save the model to disk
#index=False remove a columns with previous index
#
X_train.to_csv('./dataset/train1.csv', index=False)
X_test.to_csv('./dataset/test1.csv', index=False)

In [0]:
data_train = pd.read_csv('./dataset/train1.csv')
print(data_train.head())
print(data_train.shape)

data_test = pd.read_csv('./dataset/test1.csv')
print(data_test.head())
print(data_test.shape)

                                                text  type
0  Report Copyright Violation Trump: No Salary fo...     1
1  Your News Wire WikiLeaks Bombshell: ‘There Is ...     1
2    reports on What You Need to Know About the '...     1
3  The second video (bottom one ) proves what Tru...     1
4  “Chapo Trap House”: New Left-Wing Podcast is a...     1
(10399, 2)
                                                text  type
0  “I am a person of faith. I am a Christian. I a...     1
1  Gardasil Vaccine Given without Consent and Rui...     1
2  No Search Warrant Yet: FBI Can’t Move On Email...     1
3  Only 3 Countries Left Without a ROTHSCHILD Cen...     1
4  Waking Times \n“Does anyone even know why we e...     1
(2600, 2)


In [0]:
#swap type columns with text
col = ["type", "text"]
data_train = data_train.reindex(columns = col)
data_train.head()

,type,text
0,1,Report Copyright Violation Trump: No Salary fo...
1,1,Your News Wire WikiLeaks Bombshell: ‘There Is ...
2,1,reports on What You Need to Know About the '...
3,1,The second video (bottom one ) proves what Tru...
4,1,“Chapo Trap House”: New Left-Wing Podcast is a...


In [0]:
#swap type columns with text
col = ["type", "text"]
data_test = data_test.reindex(columns = col)
data_test.head()

,type,text
0,1,“I am a person of faith. I am a Christian. I a...
1,1,Gardasil Vaccine Given without Consent and Rui...
2,1,No Search Warrant Yet: FBI Can’t Move On Email...
3,1,Only 3 Countries Left Without a ROTHSCHILD Cen...
4,1,Waking Times \n“Does anyone even know why we e...


In [0]:
#save data
data_train.to_csv('./mergedir/train1.csv', index=False)
data_test.to_csv('./mergedir/test1.csv', index=False)

print(data_train.shape)
print(data_test.shape)

(10399, 2)
(2600, 2)


In [0]:
#---------AG news dataset
!mkdir data
!wget https://s3.amazonaws.com/fast-ai-nlp/ag_news_csv.tgz -O data/data.tgz

mkdir: cannot create directory ‘data’: File exists
--2020-03-15 17:36:58--  https://s3.amazonaws.com/fast-ai-nlp/ag_news_csv.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.160.21
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.160.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11784419 (11M) [application/x-tar]
Saving to: ‘data/data.tgz’

data/data.tgz       100%[===================>]  11.24M  38.7MB/s    in 0.3s    

2020-03-15 17:36:59 (38.7 MB/s) - ‘data/data.tgz’ saved [11784419/11784419]



In [0]:
!tar -xvzf data/data.tgz -C data/
!mv data/ag_news_csv/* data/
!rm -r data/ag_news_csv/
!rm data/data.tgz

ag_news_csv/
ag_news_csv/train.csv
ag_news_csv/readme.txt
ag_news_csv/test.csv
ag_news_csv/classes.txt


In [0]:
import pandas as pd
from tqdm import tqdm_notebook

prefix = 'data/'

In [0]:
#---train set
train_df = pd.read_csv(prefix + 'train.csv', header=None)
train_df.head()

,0,1,2
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [0]:
#1change names columns train
tdf_train = train_df.copy()
tdf_train.rename(columns={0:'type', 1:'Title', 2:'Text'},inplace=True)
tdf_train.head()

,type,Title,Text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [0]:
#2 concat columns (Title+Text) and dropout the rest
import pandas as pd
tdf_train["text"] = tdf_train["Title"] + ';' + tdf_train["Text"]
#tdf.head()
tdf_train.drop(["Title", "Text"], axis=1, inplace=True)

tdf_train.head()

,type,text
0,3,Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new..."


In [0]:
#example
print(tdf_train.iloc[[400]])
print(tdf_train.iloc[[134]])

     type                                               text
400     4  Battling Robots in Japan's Pop-Culture Tech;Th...
     type                                               text
134     4  U.S. Barred From Weakening Dolphin Rules (AP);...


In [0]:
#3convert All classes(1,2,3,4) in -> 0 (not-fake)
tdf_train.loc[tdf_train['type'] == 1, 'type'] = '0'
tdf_train.loc[tdf_train['type'] == 2, 'type'] = '0'
tdf_train.loc[tdf_train['type'] == 3, 'type'] = '0'
tdf_train.loc[tdf_train['type'] == 4, 'type'] = '0'
print(tdf_train)

       type                                               text
0         0  Wall St. Bears Claw Back Into the Black (Reute...
1         0  Carlyle Looks Toward Commercial Aerospace (Reu...
2         0  Oil and Economy Cloud Stocks' Outlook (Reuters...
3         0  Iraq Halts Oil Exports from Main Southern Pipe...
4         0  Oil prices soar to all-time record, posing new...
...     ...                                                ...
119995    0  Pakistan's Musharraf Says Won't Quit as Army C...
119996    0  Renteria signing a top-shelf deal;Red Sox gene...
119997    0  Saban not going to Dolphins yet;The Miami Dolp...
119998    0  Today's NFL games;PITTSBURGH at NY GIANTS Time...
119999    0  Nets get Carter from Raptors;INDIANAPOLIS -- A...

[120000 rows x 2 columns]


In [0]:
from collections import Counter
print(Counter(tdf_train['type'].values))

Counter({'0': 120000})


In [0]:
tdf_train.to_csv('./data/train2.csv', index=False)

In [0]:
train2 = pd.read_csv('./data/train2.csv')
print(train2.head())
print(train2.shape)

   type                                               text
0     0  Wall St. Bears Claw Back Into the Black (Reute...
1     0  Carlyle Looks Toward Commercial Aerospace (Reu...
2     0  Oil and Economy Cloud Stocks' Outlook (Reuters...
3     0  Iraq Halts Oil Exports from Main Southern Pipe...
4     0  Oil prices soar to all-time record, posing new...
(120000, 2)


In [0]:
#-------------------------

In [0]:
#---test set
test_df = pd.read_csv(prefix + 'test.csv', header=None)
test_df.head()

In [0]:
print(test_df.shape)

(7600, 3)


In [0]:
#1change names columns test
tdf_test = test_df.copy()
tdf_test.rename(columns={0:'type', 1:'Title', 2:'Text'},inplace=True)
tdf_test.head()

,type,Title,Text
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...


In [0]:
#2 concat columns (Title+Text) and dropout the rest
import pandas as pd
tdf_test["text"] = tdf_test["Title"] + ';' + tdf_test["Text"]
#tdf.head()
tdf_test.drop(["Title", "Text"], axis=1, inplace=True)

tdf_test.head()

,type,text
0,3,Fears for T N pension after talks;Unions repre...
1,4,The Race is On: Second Private Team Sets Launc...
2,4,Ky. Company Wins Grant to Study Peptides (AP);...
3,4,Prediction Unit Helps Forecast Wildfires (AP);...
4,4,Calif. Aims to Limit Farm-Related Smog (AP);AP...


In [0]:
#3convert All classes(1,2,3,4) in -> 0 (not-fake)
tdf_test.loc[tdf_test['type'] == 1, 'type'] = '0'
tdf_test.loc[tdf_test['type'] == 2, 'type'] = '0'
tdf_test.loc[tdf_test['type'] == 3, 'type'] = '0'
tdf_test.loc[tdf_test['type'] == 4, 'type'] = '0'
print(tdf_test)

     type                                               text
0       0  Fears for T N pension after talks;Unions repre...
1       0  The Race is On: Second Private Team Sets Launc...
2       0  Ky. Company Wins Grant to Study Peptides (AP);...
3       0  Prediction Unit Helps Forecast Wildfires (AP);...
4       0  Calif. Aims to Limit Farm-Related Smog (AP);AP...
...   ...                                                ...
7595    0  Around the world;Ukrainian presidential candid...
7596    0  Void is filled with Clement;With the supply of...
7597    0  Martinez leaves bitter;Like Roger Clemens did ...
7598    0  5 of arthritis patients in Singapore take Bext...
7599    0  EBay gets into rentals;EBay plans to buy the a...

[7600 rows x 2 columns]


In [0]:
from collections import Counter
print(Counter(tdf_test['type'].values))

Counter({'0': 7600})


In [0]:
tdf_test.to_csv('./data/test2.csv', index=False)

In [0]:
#print test2.csv
test2 = pd.read_csv('./data/test2.csv')
print(test2.head())
print(test2.shape)

   type                                               text
0     0  Fears for T N pension after talks;Unions repre...
1     0  The Race is On: Second Private Team Sets Launc...
2     0  Ky. Company Wins Grant to Study Peptides (AP);...
3     0  Prediction Unit Helps Forecast Wildfires (AP);...
4     0  Calif. Aims to Limit Farm-Related Smog (AP);AP...
(7600, 2)


In [0]:
#-------------------------------

In [0]:
#Merge train1.csv(fake) with train2.csv(not-fake) and test1.csv(fake) with test2.csv(not-fake)
train1 = pd.read_csv('./mergedir/train1.csv')
print(train1.head())
print(train1.shape)

train2 = pd.read_csv('./mergedir/train2.csv')
print(train2.head())
print(train2.shape)

   type                                               text
0     1  Report Copyright Violation Trump: No Salary fo...
1     1  Your News Wire WikiLeaks Bombshell: ‘There Is ...
2     1    reports on What You Need to Know About the '...
3     1  The second video (bottom one ) proves what Tru...
4     1  “Chapo Trap House”: New Left-Wing Podcast is a...
(10399, 2)
   type                                               text
0     0  Wall St. Bears Claw Back Into the Black (Reute...
1     0  Carlyle Looks Toward Commercial Aerospace (Reu...
2     0  Oil and Economy Cloud Stocks' Outlook (Reuters...
3     0  Iraq Halts Oil Exports from Main Southern Pipe...
4     0  Oil prices soar to all-time record, posing new...
(120000, 2)


In [0]:
#Concat training set
frames = [train1, train2]
train = pd.concat(frames,ignore_index=True)
print(train.shape)

(130399, 2)


In [0]:
train.head()

,type,text
0,1,Report Copyright Violation Trump: No Salary fo...
1,1,Your News Wire WikiLeaks Bombshell: ‘There Is ...
2,1,reports on What You Need to Know About the '...
3,1,The second video (bottom one ) proves what Tru...
4,1,“Chapo Trap House”: New Left-Wing Podcast is a...


In [0]:
print(train.iloc[[10398]])
print(train.iloc[[10399]])
print(train.iloc[[10400]])
print(train.iloc[[134]])

In [0]:
train.tail()

In [0]:
#shuffle rows
#train = train.reindex(np.random.permutation(train.index))
#train.head()


In [0]:
train.shape

(130399, 2)

In [0]:
train.tail()

,type,text
130394,0,Pakistan's Musharraf Says Won't Quit as Army C...
130395,0,Renteria signing a top-shelf deal;Red Sox gene...
130396,0,Saban not going to Dolphins yet;The Miami Dolp...
130397,0,Today's NFL games;PITTSBURGH at NY GIANTS Time...
130398,0,Nets get Carter from Raptors;INDIANAPOLIS -- A...


In [0]:
#Save final training set
train.to_csv('./mergedir/train.csv', index=False)

In [0]:
#Final operation test set
#
#1 merge test set
test1 = pd.read_csv('./mergedir/test1.csv')
print(test1.head())
print(test1.shape)

test2 = pd.read_csv('./mergedir/test2.csv')
print(test2.head())
print(test2.shape)

   type                                               text
0     1  “I am a person of faith. I am a Christian. I a...
1     1  Gardasil Vaccine Given without Consent and Rui...
2     1  No Search Warrant Yet: FBI Can’t Move On Email...
3     1  Only 3 Countries Left Without a ROTHSCHILD Cen...
4     1  Waking Times \n“Does anyone even know why we e...
(2600, 2)
   type                                               text
0     0  Fears for T N pension after talks;Unions repre...
1     0  The Race is On: Second Private Team Sets Launc...
2     0  Ky. Company Wins Grant to Study Peptides (AP);...
3     0  Prediction Unit Helps Forecast Wildfires (AP);...
4     0  Calif. Aims to Limit Farm-Related Smog (AP);AP...
(7600, 2)


In [0]:
#2 concat set
frames = [test1, test2]
test = pd.concat(frames, ignore_index=True)
print(test.shape)

(10200, 2)


In [0]:
test.head()

,type,text
0,1,“I am a person of faith. I am a Christian. I a...
1,1,Gardasil Vaccine Given without Consent and Rui...
2,1,No Search Warrant Yet: FBI Can’t Move On Email...
3,1,Only 3 Countries Left Without a ROTHSCHILD Cen...
4,1,Waking Times \n“Does anyone even know why we e...


In [0]:
print(test.iloc[[2599]])
print(test.iloc[[2600]])
print(test.iloc[[2601]])

      type text
2599     1     
      type                                               text
2600     0  Fears for T N pension after talks;Unions repre...
      type                                               text
2601     0  The Race is On: Second Private Team Sets Launc...


In [0]:
test.tail()

,type,text
10195,0,Around the world;Ukrainian presidential candid...
10196,0,Void is filled with Clement;With the supply of...
10197,0,Martinez leaves bitter;Like Roger Clemens did ...
10198,0,5 of arthritis patients in Singapore take Bext...
10199,0,EBay gets into rentals;EBay plans to buy the a...


In [0]:
#Save final training set
test.to_csv('./mergedir/test.csv', index=False)